In [1]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 KB 4.3 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)


In [2]:
# export.py
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    "neo4j+s://64a663c2.databases.neo4j.io",
    auth=("neo4j", "6ty6zc9rtk5ETSjQ3TUROHNv1UzNvsBHXgW8kd7WuRw")
)

def export_data():
    with driver.session() as session:
        with open('export.cypher', 'w', encoding='utf-8') as f:
            # Export nodes
            print("Exporting nodes...")
            result = session.run("MATCH (n) RETURN n, labels(n) as labels, id(n) as id")
            for record in result:
                node = record["n"]
                labels = ':'.join(record["labels"])
                props = dict(node.items())
                props_str = ', '.join([f"{k}: {repr(v)}" for k, v in props.items()])
                f.write(f"CREATE (n:{labels} {{{props_str}}});\n")
            
            # Export relationships
            print("Exporting relationships...")
            result = session.run("MATCH (a)-[r]->(b) RETURN a, r, b, type(r) as type")
            for record in result:
                a = record["a"]
                b = record["b"]
                r = record["r"]
                rel_type = record["type"]
                
                # Match nodes by properties
                a_props = dict(a.items())
                b_props = dict(b.items())
                r_props = dict(r.items())
                
                a_match = ', '.join([f"{k}: {repr(v)}" for k, v in list(a_props.items())[:2]])
                b_match = ', '.join([f"{k}: {repr(v)}" for k, v in list(b_props.items())[:2]])
                r_props_str = '{' + ', '.join([f"{k}: {repr(v)}" for k, v in r_props.items()]) + '}' if r_props else ''
                
                f.write(f"MATCH (a {{{a_match}}}), (b {{{b_match}}}) CREATE (a)-[:{rel_type} {r_props_str}]->(b);\n")

    print("Export complete: export.cypher")

export_data()
driver.close()

Exporting nodes...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=1, column=42, offset=41>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 41, 'line': 1, 'column': 42}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'MATCH (n) RETURN n, labels(n) as labels, id(n) as id'


Exporting relationships...
Export complete: export.cypher


In [5]:
import json

driver = GraphDatabase.driver(
    "neo4j+s://64a663c2.databases.neo4j.io",
    auth=("neo4j", "6ty6zc9rtk5ETSjQ3TUROHNv1UzNvsBHXgW8kd7WuRw")
)

def cypher_value(v):
    if v is None:
        return "null"
    elif isinstance(v, bool):
        return str(v).lower()
    elif isinstance(v, (int, float)):
        return str(v)
    elif isinstance(v, str):
        return json.dumps(v)
    elif isinstance(v, list):
        return '[' + ', '.join(cypher_value(x) for x in v) + ']'
    elif hasattr(v, 'isoformat'):  # datetime
        return json.dumps(v.isoformat())
    else:
        return json.dumps(str(v))

def export_data():
    with driver.session() as session:
        with open('export1.cypher', 'w', encoding='utf-8') as f:
            # Export nodes
            print("Exporting nodes...")
            result = session.run("MATCH (n) RETURN n, labels(n) as labels")
            for record in result:
                node = record["n"]
                labels = ':'.join(record["labels"]) if record["labels"] else "UnlabeledNode"
                props = {k: v for k, v in node.items()}
                props_str = ', '.join([f"{k}: {cypher_value(v)}" for k, v in props.items()])
                f.write(f"CREATE (:{labels} {{{props_str}}});\n")
            
            # Export relationships
            print("Exporting relationships...")
            result = session.run("""
                MATCH (a)-[r]->(b) 
                RETURN elementId(a) as aid, elementId(b) as bid, 
                       type(r) as type, properties(r) as props
            """)
            
            node_map = {}
            result2 = session.run("MATCH (n) RETURN elementId(n) as eid, properties(n) as props, labels(n) as labels")
            for rec in result2:
                key_props = dict(list(rec["props"].items())[:1])  # First property as key
                node_map[rec["eid"]] = (rec["labels"][0] if rec["labels"] else "UnlabeledNode", key_props)
            
            result = session.run("MATCH (a)-[r]->(b) RETURN elementId(a) as aid, elementId(b) as bid, type(r) as type, properties(r) as props")
            for record in result:
                aid = record["aid"]
                bid = record["bid"]
                rel_type = record["type"]
                r_props = record["props"]
                
                a_label, a_props = node_map.get(aid, ("UnlabeledNode", {}))
                b_label, b_props = node_map.get(bid, ("UnlabeledNode", {}))
                
                a_match = ', '.join([f"{k}: {cypher_value(v)}" for k, v in a_props.items()])
                b_match = ', '.join([f"{k}: {cypher_value(v)}" for k, v in b_props.items()])
                r_props_str = '{' + ', '.join([f"{k}: {cypher_value(v)}" for k, v in r_props.items()]) + '}' if r_props else ''
                
                f.write(f"MATCH (a:{a_label} {{{a_match}}}), (b:{b_label} {{{b_match}}}) CREATE (a)-[:{rel_type} {r_props_str}]->(b);\n")

    print("Export complete")

export_data()
driver.close()

Exporting nodes...
Exporting relationships...
Export complete
